In [1]:
import numpy as np
import matplotlib.pyplot as plt
from importlib import reload

import Detectors.Deep_Learning.AutoEncoders.utils
reload(Detectors.Deep_Learning.AutoEncoders.utils)
from Detectors.Deep_Learning.AutoEncoders.utils import SeqDataset,train_epoch,eval_data,train_model

import flow.visualize.visualize_ring as visualize_ring
reload(visualize_ring)
from flow.visualize.visualize_ring import get_measured_leader,get_rel_dist_to_measured_leader,get_vel_of_measured_leader


import torch
print(torch.__version__)

# Anti-Flow specific functions for  detection:

from Detectors.Deep_Learning.AutoEncoders.utils import sliding_window
from Detectors.Deep_Learning.AutoEncoders.cnn_lstm_ae import CNNRecurrentAutoencoder

import utils
reload(utils)
from utils import Bando_OVM_FTL

import os

print('Libraries loaded.')

import Adversaries.controllers.car_following_adversarial
from Adversaries.controllers.car_following_adversarial import FollowerStopper_Overreact
from Adversaries.controllers.car_following_adversarial import ACC_Benign

reload(Detectors.Deep_Learning.AutoEncoders.utils)
from Detectors.Deep_Learning.AutoEncoders.utils import get_cnn_lstm_ae_model

reload(utils)
from utils import assess_relative_model_on_attack

print('Libraries loaded.')

ModuleNotFoundError: No module named 'Detectors'

In [1]:
#Run with just one class of driver:


def run_benign_sim():
    driver_controller_list = []

    sim_time = 1000
    ring_length = 600

    a_mean=0.8
    b_mean=20.0
    s0_mean=1.0
    s1_mean=2.0
    Vm_mean=15.0

    for i in range(30):
        a = a_mean + np.random.normal(0,0.1)
        b = b_mean + np.random.normal(0,0.5)
        s0 = s0_mean + np.random.normal(0,0.2)
        s1 = s1_mean + np.random.normal(0,0.2)
        Vm = Vm_mean + np.random.normal(0,1.5)

        label = 'bando_ftl_ovm_a'+str(np.round(a,2))+'_b'+str(np.round(b,2))+'_s0'+str(np.round(s0,2))+'_s1'+str(np.round(s1,2))+'_Vm'+str(np.round(Vm,2))
        cfm_controller = (Bando_OVM_FTL,{'a':a,'b':b,'s0':s0,'s1':s1,'Vm':Vm,'noise':0.1})
        driver_controller_list.append([label,cfm_controller,1])


    k_1_mean = 1.0
    k_2_mean = 1.0
    h_mean = 1.5
    V_m_mean = 15.0

    for i in range(10):
        k_1 = k_1_mean + np.random.normal(0,0.2)
        k_2 = k_2_mean + np.random.normal(0,0.2)
        h = h_mean + np.random.normal(0,0.2)
        V_m = V_m_mean + np.random.normal(0,1.0)

        label = 'ACC_k_1'+str(np.round(k_1,2))+'_k_2'+str(np.round(k_2,2))+'_h'+str(np.round(h,2))+'_V_m'+str(np.round(V_m,2))
        cfm_controller = (ACC_Benign,{'k_1':k_1,'k_2':k_2,'h':h,'V_m':V_m})
        driver_controller_list.append([label,cfm_controller,1])
    
    sim_res_list = utils.run_ring_sim_variable_cfm(driver_controller_list = driver_controller_list,
                                                               ring_length = ring_length,
                                                               sim_time=sim_time)
    
    emission_path = sim_res_listp[1]
    
    return emission_path

def run_attack_sim():

    driver_controller_list = []

    sim_time = 1000
    ring_length = 600

    a_mean=0.8
    b_mean=20.0
    s0_mean=1.0
    s1_mean=2.0
    Vm_mean=15.0

    for i in range(30):
        a = a_mean + np.random.normal(0,0.1)
        b = b_mean + np.random.normal(0,0.5)
        s0 = s0_mean + np.random.normal(0,0.2)
        s1 = s1_mean + np.random.normal(0,0.2)
        Vm = Vm_mean + np.random.normal(0,1.5)

        label = 'bando_ftl_ovm_a'+str(np.round(a,2))+'_b'+str(np.round(b,2))+'_s0'+str(np.round(s0,2))+'_s1'+str(np.round(s1,2))+'_Vm'+str(np.round(Vm,2))
        cfm_controller = (Bando_OVM_FTL,{'a':a,'b':b,'s0':s0,'s1':s1,'Vm':Vm,'noise':0.1})
        driver_controller_list.append([label,cfm_controller,1])


    k_1_mean = 1.0
    k_2_mean = 1.0
    h_mean = 1.5
    V_m_mean = 15.0

    for i in range(8):
        k_1 = k_1_mean + np.random.normal(0,0.2)
        k_2 = k_2_mean + np.random.normal(0,0.2)
        h = h_mean + np.random.normal(0,0.2)
        V_m = V_m_mean + np.random.normal(0,1.0)

        label = 'ACC_k_1'+str(np.round(k_1,2))+'_k_2'+str(np.round(k_2,2))+'_h'+str(np.round(h,2))+'_V_m'+str(np.round(V_m,2))
        cfm_controller = (ACC_Benign,{'k_1':k_1,'k_2':k_2,'h':h,'V_m':V_m})
        driver_controller_list.append([label,cfm_controller,1])

    v_des = 10.0
    braking_period = 5.0
    braking_rate = -2.0

    adversary = (FollowerStopper_Overreact, {'v_des':v_des,
                                            'braking_rate':braking_rate,
                                            'braking_period':braking_period})

    label_adv = 'FStop_vd'+str(v_des)+'_bperiod'+str(braking_period)+'_brate'+str(braking_rate)
    driver_controller_list.append([label_adv,adversary,1])


    v_des = 11.0
    braking_period = 3.0
    braking_rate = -3.0

    adversary = (FollowerStopper_Overreact, {'v_des':v_des,
                                            'braking_rate':braking_rate,
                                            'braking_period':braking_period})

    label_adv = 'FStop_vd'+str(v_des)+'_bperiod'+str(braking_period)+'_brate'+str(braking_rate)
    driver_controller_list.append([label_adv,adversary,1])
    
    sim_res_list = utils.run_ring_sim_variable_cfm(driver_controller_list = driver_controller_list,
                                                               ring_length = ring_length,
                                                               sim_time=sim_time)
    emission_path = sim_res_list[1]
    
    return emission_path
    
print('Simulation functions defined.')

SyntaxError: invalid syntax (<ipython-input-1-24e7ab8f8713>, line 4)

In [ ]:
max_losses = max_losses_75_percentGPS[1]
cluster_diff = 0.1
labels,cluster_centroids = k_means_cluster(max_losses,cluster_diff=cluster_diff)

cluster_centroids = cluster_centroids*(np.max(max_losses)-np.min(max_losses))
cluster_centroids = cluster_centroids+np.max(max_losses)

fontsize=20
plt.figure(figsize=[10,5])
plt.plot(max_losses,labels,'.',markersize=10,label='Testing samples')
# plt.plot(cluster_centroids,[0,1],'*',markersize=10,label='Cluster centroids')
plt.ylabel('Assigned label',fontsize=fontsize)
plt.xlabel('Maximum reconstruction loss',fontsize=fontsize)
plt.title('Classification on max losses',fontsize=fontsize)
plt.xticks(fontsize=fontsize)
plt.yticks(fontsize=fontsize)